In [5]:
search_for = 122
start_from = 0
threads = 10
thread = 0
log_freq = 10000



#in this cell, for a given lattice, we compute all forbidden nodes, 
#i.e. those nodes whose Voronoi cell is at a distance less than doubled covering radius

#An* represented in m=n+1 as sum x_i=0 with x_i\equiv to each other mod m. 
#Forbidden nodes will be recorded w.r.t the basis (-n,1,1,..) and permutations

n=5 #lattice dim
m=n+1 #space dim




R = sqrt(n*(n+1)*(n+2)/12) #covering radius

#first we generate all nodes with norm <=4R, and with norm<=2R
t=1 #.3 #can increase a bit for random from non-forbidden
short = []
vshort = []
K = ceil(t*4*R)
K2 = (t*4*R)^2
Kv = (2*R)^2

indexes = [] #arrays of integers between -K and K having the same residue modulo m 
for rem in range(m):
    res = []
    for i in range(-K,K+1):
        if i%m==rem:
            res.append(i)
    indexes.append(res)

for c in range(0,K+1):
    for coefs in mrange_iter([indexes[c%m] for i in range(m-2)]):
        p = [c]+list(coefs)+[-c-sum(coefs)]
        is_sorted = True
        for i in range(m-1):
            if p[i]<p[i+1]:
                is_sorted = False
                break
        if is_sorted:
            norm2 = sum((p[i])^2 for i in range(m))
            if norm2>0:
                if norm2 <= Kv:
                    vshort.append(p)
                if norm2 <= K2:
                    short.append(p)           

print("very short:",len(vshort))
print("short:",len(short))


from time import time
tm = time()


#defining the cone arising due to symmetry - it suffices to construct Voronoi cell in it
ineqs = [] 
for i in range(m-1):
    toadd = [0]*(m+1)
    toadd[i+1]=1
    toadd[i+2]=-1
    ineqs.append(toadd) 
    
ineqs += [[0]+[1]*m,[0]+[-1]*m] #we are in the hyperplane where all coords sum to zero

#now Voronoi cell planes
for p in vshort: #suffices to look at nodes at most 2R
    ineqs.append([sum((p[i])^2 for i in range(m))]+[-2*p[i] for i in range(m)])

V = Polyhedron(ieqs=ineqs)
    
print(time()-tm)    

### orthogonal projection affine operator onto (improper) non-empty polytope 
zerom = matrix(m)
def projp(P):
    nv = P.n_vertices()
    z = vector(P.vertices()[0])
    if nv==1:
        return zerom,z
    A = matrix([vector(P.vertices()[i])-z for i in range(1,nv)]).transpose()
    return A*A.pseudoinverse(),z


forb = []

for d in range(0,n): #dim of face
    for ff in V.faces(d):
        proj = projp(ff)
        for p in short:
            if p not in forb:
                x = vector([t/2 for t in p]) #dist from 1/2 of a node to the Voronoi cell equals 1/2 of the dist between cells at zero and at the node
                proj_pnt = proj[0]*(x-proj[1])+proj[1]
                if proj_pnt in ff.as_polyhedron():
                    dist = (x-proj_pnt).norm()
                    if dist<R:
                        forb.append(p)

print("forbidden:",len(forb))

#expand by the basis (-n,1,1,..) and permutations
def basis_coefs(x):
    y = x[0:(m-1)]
    sy = sum(y)
    return [(-sy-y[i])/m for i in range(m-1)]

### forbd = forb with possible permutations
forbd = {}
forbdv = []
for x in forb:
    for t in Permutations(list(x)):
        c = vector(basis_coefs(t))
        if str(c) not in forbd:
            forbd[str(c)]=0
            if sum(c)>=0:
                forbdv.append(c)
print("forbidden without symmetries:",len(forbd))
###





fn = 0
for x in forb:
    norm2 = sum(t^2 for t in x)
    if norm2>fn:
        fn = norm2
print("square L2 norm of forbidden:",fn)


small = 2 #max abs value of a small coefficient
sc = []
for c in mrange_iter([range(-small,small+1) for i in range(n)]):
    if sum(c)>=0:
        sc.append(vector(c))
sc.sort(key=lambda i: sum(x*x for x in i)) #abs(x) in place of x*x seems a bit slower
sc = sc[1:] #removing zero

print("small:",len(sc))

order = [(-1)^i*(1/4+i/2)-1/4 for i in range(1000)] #[0,-1,1,-2,2,...]

def check_sublattice(basis,target=Infinity): 
    A = matrix(basis) #rows
    #A = A.LLL() #benchmark to see if helps
    A = A.transpose() #columns

#     Ad = abs(A.det())
#     if Ad==0 or Ad>=target:
#         return Infinity
        
    for coefs in sc:
        if str(A*coefs) in forbd:
            return Infinity

    Ai = A.inverse()
    Ai_norms = [sum(t^2 for t in Ai[i]) for i in range(n)]
    M = []
    Mp = 1
    Mps = 1
    for i in range(n):
        mi = floor(sqrt(fn*Ai_norms[i]))
        M.append(mi)
        Mp *= (2*mi+1)
        Mps *= (2*min(small,mi)+1)
        
    if max(M)>small: #otherwise already considered
        if Mp-Mps<len(forbd): #faster using A, otherwise A_inv
            for coefs in mrange_iter([order[0:(2*M[i]+1)] for i in range(n)]):
                if sum(coefs)>=0 and max(abs(x) for x in coefs)>small:
                    if str(A*vector(coefs)) in forbd:
                        return Infinity
        else: #check if A_inv times forbidden is not in Z^n
            for x in forbdv:
                y = Ai*x
                allint = True
                for t in y:
                    if t!=floor(t):
                        allint = False
                        break
                if allint:
                    return Infinity   
    
    return 0#Ad



#random/exhaustive sublattice search, skip to the cell starting with good for search using non-forbidden generating vectors of small norm
#each type of a matrix is the numbers on the diagonal
maxd = search_for #bound for diagonal entries
mindet = search_for #minimum determinant
maxdet = search_for #maximum determinant

def all_types(m,d1,d2,n):
    res = []
    if n==1:
        for k in range(d1,min(d2,m)+1):
            res.append([k])
        return res
    for k in range(1,min(d2,m)+1):
        prev = all_types(m,ceil(d1/k),floor(d2/k),n-1)
        for x in prev:
            res.append([k]+x)
    return res

types = all_types(maxd,mindet,maxdet,n)

print("types:",len(types))

def ind_max(t): #given type, lists maxima of indexes to define entries above the diagonal
    return [t[i]^i for i in range(1,n)]


allc = 0
for t in types:
    allc += prod(ind_max(t))
print("all cases:",allc)

from random import randint
def rand_ind(t): #given type, return random indexes
    maxs = ind_max(t)
    return [randint(0,maxs[i]-1) for i in range(n-1)]

def sublat_mat(t,ind): #given type and indexes, return sublattice basis
    res = matrix(n)
    for i in range(n):
        res[i,i] = t[i]
    for i in range(1,n):
        tt = -floor(t[i]/2)
        indc = ind[i-1]
        for j in range(i):
            res[j,i] = tt + (indc % t[i])
            indc = floor(indc / t[i])
    return res

def exh_search():
    tm = time()
    bestv = Infinity
    best = []
    allcnt = 0
    for tt in types:
        print("type:",tt,prod(tt))
        cnt = 0
        maxinds = ind_max(tt)
        cM = prod(maxinds)
        print("cases of this type:",cM)
        for i in range(cM):
            allcnt += 1
            if allcnt%(log_freq*threads)==0:
                print(allcnt,(time()-tm)/3600*(allc-allcnt)/(log_freq*threads))
                tm=time()
            if allcnt>=start_from and allcnt%threads==thread:
                inds = []
                l = i
                for j in range(n-1):
                    inds.append(l%maxinds[j])
                    l=floor(l/maxinds[j])
                mat = sublat_mat(tt,inds)
                cs = check_sublattice(mat)
                if cs==0:
                    cs = prod(tt)
                    if cs<bestv:
                        print("sublattice basis:")
                        print(mat)
                        bestv = cs
                        best = [mat]
                    elif cs==bestv:
                        best.append(mat)
        print("best so far:",len(best))
    print("all cases:",allcnt)
    print("best:",len(best))
    return best





best = exh_search()




very short: 5
short: 39
0.002406597137451172
forbidden: 31
forbidden without symmetries: 1984
square L2 norm of forbidden: 264
small: 1752
types: 25
all cases: 436374755
type: [1, 1, 1, 1, 122] 122
cases of this type: 221533456
100000 20.252934511119697
200000 31.988085958493194
300000 32.24759959043568
400000 31.129193162006814
500000 32.784889491421545
600000 31.455379232231483
700000 30.715476336133786
800000 31.163622007238565
900000 25.80233280022021
1000000 17.772078192948022
1100000 30.738105649635923
1200000 30.74233026902487
1300000 30.747107249591505
1400000 32.40876290091159
1500000 31.069428797381292
1600000 30.76321149044463
1700000 31.202994131572897
1800000 29.112521161916714
1900000 13.93719711139305
2000000 29.672184032796988
2100000 29.501124080401283
2200000 28.97489758370942
2300000 30.554662556927504
2400000 29.35331604851457
2500000 28.731458989364405
2600000 28.80739219744527
2700000 28.19886908303786
2800000 12.761729127466904
2900000 28.716471409224017
3000000 

29400000 21.884065305650775
29500000 30.62663352943556
29600000 28.737527084129752
29700000 28.603351232753884
29800000 27.079932976388804
29900000 26.41832253375895
30000000 11.973784990910804
30100000 27.092510494847332
30200000 26.037519056571828
30300000 28.18404008863422
30400000 28.534397462781318
30500000 29.761754872682094
30600000 29.482018296168462
30700000 15.236909797196025
30800000 29.13910099951803
30900000 29.467529381084372
31000000 29.20065584523062
31100000 28.581347929882995
31200000 14.873414913063392
31300000 29.235786259104795
31400000 28.04556287459269
31500000 28.469054266777672
31600000 26.812233152025364
31700000 25.13381738560788
31800000 15.189484076578665
31900000 22.479173343145234
32000000 25.125474988915524
32100000 27.451120137552568
32200000 27.529517825244856
32300000 28.618430187930016
32400000 29.573420673049906
32500000 14.978065747700997
32600000 28.524091249115177
32700000 29.279989705391866
32800000 29.423438608742874
32900000 29.94183726776112


59100000 10.639874879434153
59200000 20.877651460597313
59300000 12.487523577055006
59400000 14.029812457222278
59500000 11.659809381887644
59600000 18.713354141647745
59700000 18.57907363996147
59800000 23.77986924729527
59900000 23.05028559470288
60000000 16.092349253017638
60100000 17.570281733975374
60200000 17.491705833980976
60300000 18.18308154728486
60400000 11.637399180492505
60500000 7.378827820370397
60600000 12.109330105713093
60700000 20.294262819844732
60800000 17.709720195029597
60900000 6.49912763192606
61000000 13.361852888638982
61100000 16.60878277863175
61200000 20.14474653908488
61300000 7.828988603363231
61400000 15.260373590190103
61500000 17.549207508923285
61600000 22.442322846111782
61700000 17.48019731578618
61800000 18.93625663208929
61900000 16.046283055533284
62000000 17.70412934866381
62100000 23.323751231996408
62200000 18.19602534213055
62300000 10.99673139159136
62400000 14.137655539273327
62500000 21.15942379361959
62600000 14.179773589004972
62700000

88800000 24.861237866943004
88900000 26.91477051443316
89000000 26.99563237863043
89100000 26.33638560392729
89200000 25.716041363882244
89300000 22.866358963658552
89400000 23.730023738240934
89500000 23.42976216637426
89600000 23.770109095483264
89700000 20.225150463786168
89800000 15.46996709948851
89900000 14.735496598504067
90000000 20.82629642946969
90100000 13.20554188966806
90200000 23.478504060579628
90300000 23.084007220158014
90400000 23.84270917518288
90500000 24.514647367989834
90600000 23.058369351081673
90700000 24.56118039208925
90800000 26.846297256732257
90900000 26.196598144048878
91000000 25.51545828623686
91100000 23.078883192555814
91200000 24.50641954003999
91300000 24.331179378574767
91400000 23.80697678235964
91500000 24.418435375902135
91600000 11.496956930081225
91700000 17.896928108464092
91800000 15.657453007771233
91900000 14.087308486211272
92000000 23.090256631877025
92100000 23.18198173265773
92200000 23.824059358099735
92300000 23.92519472256947
924000

117900000 31.014561042716235
118000000 32.59979877344585
118100000 28.84866171858591
118200000 24.634018876916493
118300000 24.274905542739507
118400000 23.441837103426714
118500000 24.773467222041972
118600000 24.53481087190558
118700000 24.57686819892609
118800000 23.575278943790337
118900000 15.604090053158647
119000000 3.7373534913194484
119100000 15.856201276700878
119200000 24.45627757377498
119300000 24.631693792358508
119400000 23.283050813603783
119500000 25.64110693053292
119600000 24.012746671109674
119700000 25.97474221814456
119800000 25.14816023441492
119900000 25.411934169855627
120000000 22.391691978839546
120100000 21.914131983244555
120200000 20.418783273050767
120300000 21.310025358809725
120400000 21.43531827034972
120500000 20.253323187936186
120600000 20.151463264861356
120700000 14.431411643792082
120800000 3.9891729810187715
120900000 11.16533359569518
121000000 21.217789779655842
121100000 21.48099255757032
121200000 21.261885968878808
121300000 21.761626805777

...

320300000 4.989102147363782
320400000 5.030274506027361
320500000 2.1817775975728066
320600000 2.5496639918968085
320700000 5.22912039694348
320800000 5.35882277116497
320900000 5.592793609497428
321000000 5.598682732720352
321100000 5.667163968296286
321200000 6.007998834401875
321300000 5.463187279945692
321400000 5.908636972910693
321500000 5.2729727727010545
321600000 2.80330254192297
321700000 4.357797008316673
321800000 5.446520131869625
321900000 5.058905623801308
322000000 5.263416222533818
322100000 5.031099260317444
322200000 4.8182386790383624
322300000 2.96878589089035
322400000 1.605749965710749
322500000 5.203914052332897
322600000 5.831322560728327
322700000 5.416230413405279
322800000 5.214996648116058
322900000 5.802322930574708
323000000 5.952282264530093
323100000 5.009421052015649
323200000 5.894581299927957
323300000 5.459079631230384
323400000 2.559900483234452
323500000 4.632823608852956
323600000 4.845295223373214
323700000 4.78900287022646
323800000 5.169542101

348500000 2.8523383193684917
348600000 3.6770802113503187
348700000 3.868122263649123
348800000 4.169866589501996
348900000 3.167225486970299
349000000 3.5993060383238435
349100000 3.002248870669186
349200000 2.7984726736848584
349300000 3.5993008481125104
349400000 2.6788178025956353
349500000 3.74193168400534
349600000 4.087698253326568
349700000 3.851040381326327
349800000 2.9045949726913105
349900000 3.4312397739200895
350000000 3.37689590601021
350100000 2.4081623975687028
350200000 3.5376110341127744
350300000 2.6560047890921137
350400000 3.648164752603236
350500000 3.8752830711631874
350600000 3.9714117689684825
350700000 2.863578027036409
350800000 3.5685653493322333
350900000 3.269794589575201
351000000 2.445786322011408
351100000 3.505197076488657
351200000 2.4458091568262477
351300000 3.7579262661752137
351400000 4.069550774153667
351500000 3.779828395034452
351600000 3.1892630051154294
351700000 3.3772640198227006
351800000 3.5559987735684513
351900000 2.0500031992038554
35

377200000 1.708046360616386
377300000 0.9322060437959631
377400000 1.9558507955992255
377500000 1.546766231586251
377600000 1.3237769079132675
377700000 1.2826258792252507
377800000 1.2484203016361561
377900000 1.633372941711068
378000000 1.2077053332818415
378100000 1.3382767425158324
378200000 0.718756598150134
378300000 1.1790045787804226
378400000 0.9576436723531253
378500000 1.6743433276212314
378600000 1.934185185807407
378700000 2.1463340976181393
378800000 2.0777556845080554
378900000 1.5972103878642188
379000000 2.1440615524977145
379100000 0.7155076693901319
379200000 1.7612398341339166
379300000 0.9061896342280209
379400000 1.8898460606647827
379500000 1.6372447005832491
379600000 2.271866605967237
379700000 1.9014648029495178
379800000 1.0118532538750535
379900000 1.235905187384271
380000000 0.7078326809708972
380100000 1.1876933909423717
380200000 1.1895557063036992
380300000 1.8357043322098354
380400000 2.0788703642148376
380500000 1.6681807926117413
380600000 1.167746906

404400000 0.41497930077923667
404500000 0.5481157206545902
404600000 0.6781411362298528
404700000 0.6255486070161925
404800000 0.49912101835020056
404900000 0.4077660403375924
405000000 0.7825850046132505
405100000 0.6878247125271989
405200000 0.6711807389441231
405300000 0.5779994038599895
405400000 0.8419278162319859
405500000 0.6407887397680978
405600000 0.6204133803155754
405700000 0.5654640004720357
405800000 0.4486565653012097
405900000 0.5965842335089611
406000000 0.6515257512486047
406100000 0.5901640361767808
406200000 0.3780191871000297
406300000 0.5526664271088773
406400000 0.8090031654800176
406500000 0.667769330521706
406600000 0.8573301312944062
406700000 0.4286605488376916
406800000 0.6254018155889643
406900000 0.4073479801754256
407000000 0.1281301044686536
407100000 0.13007616318643092
407200000 0.12970007856379615
407300000 0.1297989431559841
407400000 0.12581231101074153
407500000 0.12887054909347162
407600000 0.1275945277864635
407700000 0.1271083358459837
407800000

430600000 0.08101599336992368
430700000 0.09692927479735017
430800000 0.09319159254976445
430900000 0.07514644719761279
431000000 0.0997363358065188
431100000 0.06864610168737836
431200000 0.09836449173050126
431300000 0.06863461896706621
431400000 0.11215277715532977
431500000 0.06965189218567146
431600000 0.08051653416910769
431700000 0.0832577069009443
431800000 0.06639221886297067
431900000 0.0859462728678584
432000000 0.059139796612123645
432100000 0.08526187797771229
432200000 0.060511903834323096
432300000 0.07380684363962213
432400000 0.06373529574350184
432500000 0.062356777777940034
432600000 0.06912512813978725
432700000 0.05271453882388936
432800000 0.06657771354192496
432900000 0.06315363513948188
433000000 0.06055719431886077
433100000 0.04578437108120323
433200000 0.053562048520866365
433300000 0.052055217244575416
433400000 0.0577419456795156
433500000 0.051500085470212835
433600000 0.04320861040867369
433700000 0.04506520017170906
433800000 0.04041742919139067
43390000